In [ ]:
!pip install datasets

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import (BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives,
                                       TrueNegatives, Precision, Recall, AUC, binary_accuracy,Accuracy,
                                       TopKCategoricalAccuracy, CategoricalAccuracy,SparseCategoricalAccuracy)
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer,TFDebertaForSequenceClassification,DebertaTokenizerFast)
import tensorflow as tf

In [ ]:
dataset = load_dataset("csv", data_files = "/content/20000-~1.CSV")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent'],
        num_rows: 21534
    })
})

In [ ]:
dataset['train'][0]

{'flags': 'BILC',
 'utterance': "I don't have an online account, what do I have to do to register?",
 'category': 'ACCOUNT',
 'intent': 'create_account'}

In [ ]:
intents=list(set(dataset['train']['intent']))
dict_intents={intents[i]: i for i in range(len(intents))}
print(dict_intents)
print(len(intents))

{'review': 0, 'registration_problems': 1, 'get_invoice': 2, 'check_invoices': 3, 'check_cancellation_fee': 4, 'set_up_shipping_address': 5, 'newsletter_subscription': 6, 'get_refund': 7, 'recover_password': 8, 'track_refund': 9, 'cancel_order': 10, 'change_order': 11, 'delivery_period': 12, 'check_payment_methods': 13, 'complaint': 14, 'payment_issue': 15, 'change_shipping_address': 16, 'switch_account': 17, 'place_order': 18, 'edit_account': 19, 'delivery_options': 20, 'check_refund_policy': 21, 'create_account': 22, 'delete_account': 23, 'track_order': 24, 'contact_human_agent': 25, 'contact_customer_service': 26}
27


In [ ]:
def categorical_labeling(dataset):
    return {
        'utterance': dataset['utterance'],
        'intent': dict_intents[dataset['intent']]
    }

In [ ]:
prep_data = dataset.map(categorical_labeling)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [ ]:
prep_data['train'][0]

{'flags': 'BILC',
 'utterance': "I don't have an online account, what do I have to do to register?",
 'category': 'ACCOUNT',
 'intent': 22}

In [ ]:
model_id = 'microsoft/deberta-base'
tokenizer = DebertaTokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenizer_func(dataset):
    return tokenizer(dataset['utterance'],)

In [ ]:
tokenized_data = prep_data.map(tokenizer_func)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21534
    })
})

In [ ]:
tokenized_data['train'][0]

{'flags': 'BILC',
 'utterance': "I don't have an online account, what do I have to do to register?",
 'category': 'ACCOUNT',
 'intent': 22,
 'input_ids': [1,
  100,
  218,
  75,
  33,
  41,
  804,
  1316,
  6,
  99,
  109,
  38,
  33,
  7,
  109,
  7,
  5124,
  116,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors = 'tf')

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
BATCH_SIZE = 16

In [ ]:
tf_data = tokenized_data['train'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'intent'],
    shuffle = True,
    batch_size = BATCH_SIZE,
    collate_fn = data_collator
)

In [ ]:
def swap_position(dataset):
    return {
        'input_ids': dataset['input_ids'],
        'attention_mask': dataset['attention_mask']
    }, dataset['intent']

In [ ]:
tf_data = tf_data.map(swap_position)

In [ ]:
train_dataset=tf_data.take(int(0.9*len(tf_data)))
val_dataset=tf_data.skip(int(0.9*len(tf_data)))

In [ ]:
for i in train_dataset.take(1):
    print(i)

({'input_ids': <tf.Tensor: shape=(16, 18), dtype=int64, numpy=
array([[    1,   118,   236,   335,     6,   141,   109,   939,  2142,
           41,  1047,     7,  3653,   544,   116,     2,     0,     0],
       [    1,  8569,   109,   939,  2870,    10,  2267,  3674,   116,
            2,     0,     0,     0,     0,     0,     0,     0,     0],
       [    1,  9178,    64,    38,   192,    41, 38199,   116,     2,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    1,   118,   240,    41,  1316,     6,   141,    64,   939,
         1045,    65,   116,     2,     0,     0,     0,     0,     0],
       [    1, 31480,    59,   562,   103, 12259,   139,  6355,     2,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    1,   100,    33,    10,   864,     6,    64,    47,  1067,
            7, 19458,  1841,   116,     2,     0,     0,     0,     0],
       [    1, 37111,     6,  1137,   162,   114,    38,   115,  1045,
        

## Using pre-trained TFDeberta

In [ ]:
model = TFDebertaForSequenceClassification.from_pretrained(model_id, num_labels = len(intents))
model.summary()

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_deberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deberta (TFDebertaMainLaye  multiple                  138601728 
 r)                                                              
                                                                 
 pooler (TFDebertaContextPo  multiple                  590592    
 oler)                                                           
                                                                 
 cls_dropout (TFDebertaStab  multiple                  0         
 leDropout)                                                      
                                                                 
 classifier (Dense)          multiple                  20763     
                                                                 
Total params: 139213083 (531.06 MB)
Trainable params: 139213083 (531.06 MB)
Non-trainable para

In [ ]:
num_epochs = 3
batches_per_epoch = len(tokenized_data['train']) // BATCH_SIZE
total_train_step = int(batches_per_epoch * num_epochs)

In [ ]:
optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = total_train_step
)

In [ ]:
model.compile(
    optimizer = optimizer,
    metrics = ["accuracy"]
)

In [ ]:
train_history = model.fit(train_dataset, validation_data = val_dataset, epochs = 3, verbose = 1)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


1211/1211 [==============================] - 432s 252ms/step - loss: 0.2722 - accuracy: 0.9399 - val_loss: 0.0314 - val_accuracy: 0.9893
Epoch 2/3
1211/1211 [==============================] - 284s 234ms/step - loss: 0.0085 - accuracy: 0.9989 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 3/3
1211/1211 [==============================] - 281s 232ms/step - loss: 0.0031 - accuracy: 0.9998 - val_loss: 8.2609e-04 - val_accuracy: 1.0000


In [ ]:
inputs = tokenizer(["i want to track my order"
                    ], padding=True,return_tensors="tf")

logits = model(**inputs).logits
outputs=tf.argmax(logits,axis=-1).numpy()

In [ ]:
print(outputs)

[24]


In [ ]:
reverse_dict_intents={i:intents[i] for i in range(len(intents))}
print(reverse_dict_intents)

{0: 'review', 1: 'registration_problems', 2: 'get_invoice', 3: 'check_invoices', 4: 'check_cancellation_fee', 5: 'set_up_shipping_address', 6: 'newsletter_subscription', 7: 'get_refund', 8: 'recover_password', 9: 'track_refund', 10: 'cancel_order', 11: 'change_order', 12: 'delivery_period', 13: 'check_payment_methods', 14: 'complaint', 15: 'payment_issue', 16: 'change_shipping_address', 17: 'switch_account', 18: 'place_order', 19: 'edit_account', 20: 'delivery_options', 21: 'check_refund_policy', 22: 'create_account', 23: 'delete_account', 24: 'track_order', 25: 'contact_human_agent', 26: 'contact_customer_service'}


In [ ]:
for i in outputs:
    print(reverse_dict_intents[i])

track_order


In [ ]:
responses = {
        'place_order': "Sure! To place an order, you can follow these steps:\n1. Browse through our products and select the items you want to purchase.\n2. Add the selected items to your cart.\n3. Proceed to checkout and fill in your shipping and payment details.\n4. Review your order and confirm the purchase.",
        'change_shipping_address': "To change your shipping address, please follow these steps:\n1. Log in to your account.\n2. Go to the 'My Account' section.\n3. Select 'Shipping Address'.\n4. Update your address and save the changes.",
        'contact_human_agent': "If you would like to speak to a human agent, please reach out to our customer support team through our helpline at [phone number] or via email at [email address]. Our agents are available to assist you.",
        'create_account': "To create an account, follow these steps:\n1. Visit our website and click on the 'Sign Up' or 'Create Account' button.\n2. Fill in the required information such as your name, email address, and password.\n3. Verify your email address by clicking on the verification link sent to your inbox.\n4. Once verified, your account will be created successfully.",
        'edit_account': "To edit your account details, please follow these steps:\n1. Log in to your account.\n2. Navigate to the 'My Account' section.\n3. Select the 'Edit Profile' option.\n4. Update the necessary information such as your name, email, or password.\n5. Save the changes to update your account details.",
        'check_payment_methods': "To check your payment methods, follow these steps:\n1. Log in to your account.\n2. Go to the 'Payment Methods' section.\n3. Here, you can view and manage your saved payment methods.",
        'change_order': "To change an existing order, please follow these steps:\n1. Log in to your account.\n2. Go to the 'Orders' or 'Order History' section.\n3. Find the order you wish to change and select the 'Change Order' option.\n4. Follow the prompts to make the necessary changes to your order.",
        'registration_problems': "If you're experiencing issues with registration, please ensure that you're providing valid information and meeting any requirements such as password strength or email format. If the issue persists, please reach out to our customer support team for assistance.",
        'get_invoice': "To get an invoice for your order, follow these steps:\n1. Log in to your account.\n2. Go to the 'Orders' or 'Order History' section.\n3. Find the order for which you need an invoice.\n4. Select the option to 'View Invoice' or 'Download Invoice'.",
        'delivery_options': "We offer various delivery options including standard, expedited, and same-day delivery. During checkout, you can select the delivery option that best suits your needs.",
        'payment_issue': "If you're experiencing issues with payment, please ensure that your payment method is valid and has sufficient funds. You can also try using a different payment method or contacting your bank for further assistance.",
        'track_refund': "To track the status of your refund, follow these steps:\n1. Log in to your account.\n2. Go to the 'Refunds' or 'Refund Status' section.\n3. Here, you can view the status of your refund and any associated details.",
        'newsletter_subscription': "To subscribe to our newsletter, please provide your email address in the newsletter subscription section on our website. You'll receive updates, promotions, and exclusive offers straight to your inbox.",
        'cancel_order': "To cancel an order, please follow these steps:\n1. Log in to your account.\n2. Go to the 'Orders' or 'Order History' section.\n3. Find the order you wish to cancel and select the 'Cancel Order' option.\n4. Follow the prompts to confirm the cancellation.",
        'complaint': "If you have a complaint, we're here to help. Please reach out to our customer support team and provide details about your issue. We'll investigate the matter and work towards a resolution.",
        'set_up_shipping_address': "To set up your shipping address, follow these steps:\n1. Log in to your account.\n2. Navigate to the 'My Account' section.\n3. Select 'Shipping Address'.\n4. Add a new address or edit an existing one.\n5. Save your changes to set up your shipping address.",
        'recover_password': "If you've forgotten your password, follow these steps to recover it:\n1. Go to the login page on our website.\n2. Click on the 'Forgot Password' or 'Reset Password' link.\n3. Enter your email address associated with your account.\n4. Check your email for instructions on resetting your password.",
        'delete_account': "To delete your account, please contact our customer support team and request an account deletion. We'll guide you through the process and ensure that your account is deleted securely.",
        'review': "We appreciate your feedback! To leave a review, navigate to the product page of the item you purchased. Scroll down to the reviews section and click on 'Write a Review'. Share your experience and rate the product to help others make informed decisions.",
        'check_invoices': "To check your invoices, follow these steps:\n1. Log in to your account.\n2. Go to the 'Invoices' or 'Invoice History' section.\n3. Here, you can view and download your invoices for past orders.",
        'switch_account': "To switch accounts, simply log out of your current account and log in with the credentials of the account you wish to switch to.",
        'get_refund': "To request a refund, please follow these steps:\n1. Log in to your account.\n2. Go to the 'Refunds' or 'Request Refund' section.\n3. Select the order for which you want a refund.\n4. Follow the prompts to request a refund.",
        'contact_customer_service': "If you need assistance, please don't hesitate to contact our customer service team. You can reach us via phone, email, or live chat support available on our website.",
        'track_order': "To track your order, follow these steps:\n1. Log in to your account.\n2. Go to the 'Orders' or 'Order History' section.\n3. Find the order you want to track and select the 'Track Order' option.\n4. Here, you can view the current status and tracking information for your order.",
        'check_cancellation_fee': "If you're considering canceling your order, please review our cancellation policy for any applicable fees. You can find this information in the terms and conditions or contact our customer support team for assistance.",
        'delivery_period': "The delivery period depends on your location and the shipping method selected during checkout. You can check the estimated delivery time frame for your order on the product page or during checkout.",
        'check_refund_policy': "Before requesting a refund, please review our refund policy to understand eligibility criteria, processing times, and any applicable fees. You can find this information on our website or contact our customer support team for clarification.",
    }

In [ ]:
# Mapping predicted intents to predefined responses
predicted_responses = [responses[reverse_dict_intents[i]] for i in outputs]

# Printing the predicted responses
for response in predicted_responses:
    print(response)


To track your order, follow these steps:
1. Log in to your account.
2. Go to the 'Orders' or 'Order History' section.
3. Find the order you want to track and select the 'Track Order' option.
4. Here, you can view the current status and tracking information for your order.


In [ ]:
# Save the model
model.save('/content/model')



In [36]:
# Define the directory where you want to save the model
save_directory = "/content/DeBERTa_model"

# Save the model
model.save_pretrained(save_directory)